In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time



In [2]:
# Import API key
from config import g_key

In [3]:
weather_data = pd.read_csv("Desktop/python-api-challenge/output_data/cities.csv")
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Constitución,18,CL,1649432370,67,-35.3333,-72.4167,65.57,8.90
1,Qaanaaq,11,GL,1649432370,73,77.4840,-69.3632,-1.41,6.49
2,Talcahuano,75,CL,1649432370,82,-36.7167,-73.1167,60.48,6.91
3,Okhotsk,100,RU,1649432370,89,59.3833,143.3000,19.29,6.17
4,Kalmar,40,SE,1649432371,84,56.6616,16.3616,43.14,16.11
...,...,...,...,...,...,...,...,...,...
565,Bereket,61,TM,1649432444,39,39.2446,55.5154,70.74,4.21
566,Moroni,40,KM,1649432445,81,-11.7022,43.2551,81.30,8.01
567,Central Point,75,US,1649432445,67,42.3760,-122.9164,55.42,3.44
568,Upata,78,VE,1649432445,36,8.0086,-62.3989,91.74,7.81


In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [5]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))